## 성능점검 추출 코드

In [1]:
import csv
import time
import re
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 드라이버 생성시 옵션을 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

print("작업 완료")

작업 완료


In [ ]:
MAX_FETCH_COUNT = 25600
SPLIT_CSV = False
SPLIT_INDEX = 1
INPUT_FILE_PATH = "../export/vehicles_price.csv"
OUTPUT_FILE_PATH = "../export/vehicles_status.csv"

In [3]:
def main():
    try:
        if SPLIT_CSV:
            file_name = f"{SPLIT_INDEX}_{INPUT_FILE_PATH}"
        else:
            file_name = INPUT_FILE_PATH
        df = pd.read_csv(file_name)
        ids = df['id'].tolist()
        output_file_name = OUTPUT_FILE_PATH
        fetch_insurance_history(driver, ids, output_file_name)
    finally:
        driver.quit()

def fetch_insurance_history(driver, vehicle_ids, output_file):
    fieldnames = ["id", "transmission", "warranty", "mileage", "tuning_history",
                  "special_history", "usage_change", "accident", "little_accident"]
    data = []
    count = 0
    log_count = 0
    for i, vid in enumerate(vehicle_ids):
        try:
            log_count += 1
            if log_count >= 20: 
                print(f"\n누적 조회 수: {len(vehicle_ids)}개 중 {i + 1}개\n")
                log_count = 0
                
            driver.get(f"https://fem.encar.com/cars/report/inspect/{vid}")
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/ul[2]/li[1]/span[2]")))
            details = {
                "id": vid,
                "transmission": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][1]/li[1]/span[2]").text,
                "warranty": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][1]/li[3]/span[2]").text,
                "mileage": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[2]/span[2]").text,
                "tuning_history": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[4]/span[2]").text,
                "special_history": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[5]/span[2]").text,
                "usage_change": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][2]/li[6]/span[2]").text,
                "accident": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][3]/li[1]/span[2]").text,
                "little_accident": driver.find_element(By.XPATH, "//ul[@class='ReportInspectList_list_inspect__VRO1O'][3]/li[2]/span[2]").text
            }
            data.append(details)
            count += 1
            
            if(count > MAX_FETCH_COUNT) : 
                print(f"{count}개 성능 점검 데이터 추출 완료")
                break;
        except Exception:
            print(f"ID '{vid}': 판매자가 성능 점검 기록을 공개하지 않았습니다.")
            
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
        print(f"{count}개 성능 점검 데이터가 저장되었습니다.")

if __name__ == "__main__":
    main()


ID '38174011': 판매자가 성능 점검 기록을 공개하지 않았습니다.
ID '38317886': 판매자가 성능 점검 기록을 공개하지 않았습니다.

누적 조회 수: 48개 중 20개

ID '38257511': 판매자가 성능 점검 기록을 공개하지 않았습니다.
ID '37436440': 판매자가 성능 점검 기록을 공개하지 않았습니다.

누적 조회 수: 48개 중 40개

ID '38597755': 판매자가 성능 점검 기록을 공개하지 않았습니다.
43개 성능 점검 데이터가 저장되었습니다.
